In [12]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

GRAPH_NAME = "cpi_stick_flex"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [13]:
raw_df = pd.read_excel(
    ut.get_repo_root() / "code" / "data" / "stickyprice_atl.xlsx")

raw_df = raw_df.set_index(pd.to_datetime(raw_df["Date"]))

raw_df = raw_df[['12mo', '12mo.3']]

filt_df = raw_df[raw_df.index >= pd.Timestamp('2023-01-01')]

filt_df = filt_df.rename(columns={'12mo.3': 'Sticky Price CPI', '12mo': 'Flexible Price CPI'})

In [14]:
fig = go.Figure()

for col in filt_df.columns:
    fig.add_trace(
        go.Scatter(
            x=filt_df.index,
            y=filt_df[col],
            name=col,
            mode='lines',
        )
    )


# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat="0.2f%",
    ticksuffix="%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
)

fig.update_layout(
    title=dict(text='Sticky vs Flexible Price CPI'),
    xaxis_title='Date',
    yaxis_title='Change from Year Ago (%)',
)


# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")

fig.show()
